In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader,Dataset, random_split


In [3]:
if torch.cuda.is_available():
    device=torch.device(type="cuda",index=0)
else:
    device=torch.device(type="cpu",index=0)
print(device)


cpu:0


In [4]:
class CustomDataset(Dataset):
    def __init__(self, root, data, transformations = None):
        
        self.transformations = transformations
        self.im_paths = sorted(glob(f"{root}/{data}/*/*"))
        
        self.cls_names, self.cls_counts, count, data_count = {}, {}, 0, 0
        for idx, im_path in enumerate(self.im_paths):
            class_name = self.get_class(im_path)
            if class_name not in self.cls_names: 
                self.cls_names[class_name] = count
                self.cls_counts[class_name] = 1
                count += 1
            else: 
                self.cls_counts[class_name] += 1
        
    def get_class(self, path): 
        return os.path.dirname(path).split("/")[-1]
    
    def __len__(self): 
        return len(self.im_paths)

    def __getitem__(self, idx):
        
        im_path = self.im_paths[idx]
        im = Image.open(im_path).convert("RGB")
        gt = self.cls_names[self.get_class(im_path)]
        
        if self.transformations is not None: 
            im = self.transformations(im)
        
        return im, gt


In [5]:
t= transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [6]:
root ="./archive/Facial_images_dataset_for_Autism_Detection"


: 

In [7]:
from PIL import Image
import matplotlib.pyplot as plt

# Path to the image file
image_path = "./archive/Facial_images_dataset_for_Autism_Detection/train/autistic/0001.jpg"

# Read the image file using PIL
image = Image.open(image_path)
width, height = image.size

print("Image dimensions: {} x {}".format(width, height))
# Display the image using plt.imshow()
plt.imshow(image)
plt.show()


Image dimensions: 470 x 590


In [ ]:
train_data = CustomDataset(root = root, data = "train", transformations = t)
valid_data = CustomDataset(root = root, data = "valid",transformations = t)
test_data = CustomDataset(root = root, data = "test",transformations = t)


In [ ]:
ns=4
bs=32


In [ ]:
tr_dl = DataLoader(train_data, batch_size = bs, shuffle = True, num_workers = ns) 
val_dl =DataLoader(valid_data, batch_size = bs, shuffle = False, num_workers = ns)
ts_dl=DataLoader(test_data, batch_size = 1, shuffle = False, num_workers = ns)


In [ ]:
print(len(tr_dl)); print(len(val_dl)); print(len(ts_dl)); print(train_data.cls_names)


In [ ]:
def denormalize(image_tensor, mean, std):
    """Denormalize a tensor of images."""
    # Clone the tensor to avoid modifying the original tensor
    denormalized_tensor = image_tensor.clone()
    for t, m, s in zip(denormalized_tensor, mean, std):
        t.mul_(s).add_(m)  # Multiply by std and add mean
    return denormalized_tensor

def plot_images(dataset, num_images=5):
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        image, label = dataset[i]
        
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        denormalized_out = denormalize(image, mean, std)

        denormalized_out = denormalized_out.numpy()
        denormalized_out = np.transpose(denormalized_out, (1, 2, 0))
        class_name = list(dataset.cls_names.keys())[list(dataset.cls_names.values()).index(label)]
        plt.subplot(1, num_images, i+1)
        plt.imshow(denormalized_out)  
        plt.title(class_name)
        plt.axis('off')
    plt.show()

plot_images(train_data)
